<a href="https://colab.research.google.com/github/MattHandzel/ConvertImageToText/blob/main/MakingAnyImageOutOfText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Input the path to your font
fontPath = "/content/Cousine-Regular.ttf"

if fontPath == None:
  fontPath = input("What is the path to your font?\n")
  if fontPath == None:
    raise Exception("HOMIE SELECT A PATH FOR YOUR FONT")

In [ ]:
# Imports
from PIL import Image, ImageDraw, ImageFont
import zipfile
from google.colab import drive
drive.mount("/content/drive")
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import structural_similarity as ssim
import cv2
import time
import IPython.display

In [ ]:
### DEFINING FUNCTIONS THAT WILL BE USED LATER

### MSE WAS STOLE FROM SOMEWHERE
def mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err

def grayscale(nparray):
  return np.array(0.299 * nparray[:,:,0] + 0.587 * nparray[:,:,1] + 0.114 * nparray[:,:,2])

def exclude(array, removeArray):
  i = 0
  while i < len(array):
    for item in removeArray:
      if array[i] == item:
        array.pop(i)
        i-=1
    i+=1
  return array

def getBool(string):
  while True:
    inputFromUser = input(string).upper()
    trues = ["Y","YES","SURE","TAK","OUI","OUAIS","HELl YEAH", "YEAH", "WHATEVER", "AWW HECK YEAH", "HECK YEAH", "OKAY","FINE"]
    falses = ["N", "NO", "NAH", "NIE", "NON", "NAWW", "HELL NO", "AWW HECK NO", "HECK NO"]
    if inputFromUser in trues:
      return True
    elif inputFromUser in falses:
      return False
    print("Please input Y for Yes and N for No")

In [ ]:
### CONVERTING CHARACTERS TO IMAGES 

# Get the font
fnt = ImageFont.truetype(fontPath, 14)

# Start and end indexs for characters
startIndex = 11
endIndex = 20000

# Hold the image of how the character looks like
pieces = []

# Excludes some numbers because for some reason it breaks the system
excludeThese = []

totalTextSize = [0,0]
allTextSizes = []

# Getting the max height of the font so that we can do this more easier
for i in range(startIndex,endIndex):  
  letter = chr(i)
  img = Image.new('RGB', (100,maxHeight), color = (0, 0, 0))
  d = ImageDraw.Draw(img)
  textSize = list(d.textsize(letter, font = fnt))
  totalTextSize = [totalTextSize[0] + textSize[0], max([totalTextSize[1],textSize[1]])]
  allTextSizes.append(textSize)

maxHeight = totalTextSize[1]

# For each character in the range find its size and how it looks like as an image
for i in range(startIndex,endIndex):  
  letter = chr(i)
  img = Image.new('RGB', (100,maxHeight), color = (0, 0, 0))
  d = ImageDraw.Draw(img)
  textSize = list(d.textsize(letter, font = fnt))
  img = Image.new('RGB', (textSize[0],maxHeight), color = (0,0,0))
  d = ImageDraw.Draw(img)
  totalTextSize = [totalTextSize[0] + textSize[0], max([totalTextSize[1],textSize[1]])]
  allTextSizes.append(textSize)
  d.text(text = letter, xy = (0,0), font = fnt)
  try:
    pieces.append(grayscale(np.array(img, dtype = float) / 255))
  except:
    excludeThese.append(i)
  
sizeOfSpace = list(d.textsize(" ", font = fnt))

img = Image.new("RGB", totalTextSize, color = (0,0,0))
d = ImageDraw.Draw(img)
allCharacters = (" ".join([chr(i) for i in range(startIndex, endIndex)]))

# Make dictionaries which will be used later
dictionaryLetters = dict(zip(exclude([i for i in range(startIndex, endIndex)], excludeThese), allCharacters))
dictionarySizes = dict(zip(exclude([i for i in range(startIndex, endIndex)], excludeThese), allTextSizes))
letterSizes = dict(zip(exclude([i for i in range(startIndex, endIndex)], excludeThese), [z.tolist() for z in np.array([np.array(allTextSizes, dtype = int).transpose()[0], np.ones(len(allTextSizes)) * maxHeight], dtype = int).transpose()]))
pieces = dict(zip(exclude([i for i in range(startIndex, endIndex)], excludeThese), pieces))

### REMOVING ANY COPIES OF CHARACTERS SO THAT WE DONT WASTE TIME
thingsToPop = []
key_1 = list(pieces.keys())[0]
while key_1 < list(pieces.keys())[-1]:
  thingsToPop = []
  keys_2 = list(pieces.keys())
  startIndex = keys_2.index(key_1)
  for key_2 in keys_2[startIndex+1:]:
    if pieces[key_1].shape[1] == pieces[key_2].shape[1]:
      if mse(pieces[key_1], pieces[key_2]) <= 0.0001:
        thingsToPop.append(key_2)
  for thing in thingsToPop:
    pieces.pop(thing)
  keys = list(pieces.keys())
  key_1 = keys[keys.index(key_1)+1]

In [ ]:
### CONVERT THE IMAGES TO TEXT
### NOTE: IT MIGHT NOT LOOK CORRECT IN GOOGLE COLAB IF YOU'RE NOT USING A MONOSPACE FONT, COPY AND PASTE IT WITH THE FONT YOU WANT AND SEE THE RESULTS

# Path of image you want to convert
#imgPath = "/content/drive/MyDrive/pepeTheFrog.jpg"

### OR
# Upload file directly
print("CHOOSE AN IMAGE THAT YOU WANT TO CONVERT:")
imgPath = (list(google.colab.files.upload().keys())[0])
# Clears output so its easier to read
display.clear_output()

getEdges = getBool("Do you want to get the edges of your image (where there is a high difference in contrast)?\n(This is best for images that you want to be able to see the shape vs. the difference of black and white)\n")
# Clears output so its easier to read
display.clear_output()

# Using SSIM for more complex structures yields a better result
useSSIM = getBool("Do you want to use SSIM instead of MSE? (This will take much longer but will yield better results, especially if your image has a defined structure)\n")
# Clears output so its easier to read
display.clear_output()

# If im sending to a snapchat text then it uses two spaces instead of one, this makes the text image look normal
sendingToSnapchat = False

if getEdges == True:
  # If we want to get the edges/areas where there is high contrast
  img = np.array(cv2.Canny(np.array(Image.open(imgPath).convert("L")), 100, 200, 3, L2gradient=True))
else:
  # Opens the image in grayscale
  img = np.array(Image.open(imgPath).convert("L"))

if len(img.shape) > 2:
  if img.shape[2] > 1:
    figure, axes = plt.subplots(ncols=img.shape[2])
    for i in range(img.shape[2]):
      axes[i].imshow(img[:,:,i])
    figure.show()
    plt.show(plt.gcf())
    time.sleep(1)
    dimensionToUse = int(input("The image has multiple dimensions, which would you like to use?\n"))
    img = img[:,:,dimensionToUse]


# Convert to float (for calcualtions in the future)
if img.dtype == np.uint8:
  img = img.astype(np.float)
  img /= 255


# Show the image
print("CONVERTING THIS IMAGE:")
plt.imshow(img)
plt.show()

imgSize = [(img.shape[0] // maxHeight) * maxHeight, img.shape[1]]

img = img[:imgSize[0], :imgSize[1]]

string = ""

# The loops for both blocks of the if-else loop through the image, going line by line, 
# comparing how close the image portion is to an image of the character
if useSSIM == True:
  while True:
    print(round((1-(img.shape[0] / imgSize[0])) * 100),"% Done")
    if img.shape[0] < maxHeight:
      break
    x = 0
    while True:
      mses = []
      # If it is basically an empty space then we skip the trying to find what character it is and assume its a space, this makes it much faster 
      if np.sum(img[:maxHeight, x:x+maxHeight]) < 0.01:
        string += " "
        x += dictionarySizes[32][0]
      else:
        for key in (dictionarySizes.keys()):
          # Instead of making it into maxHeight by 8 slices, we get the size of the dictionary because some fonts have character being different sizes
          # and this corrects for that difference so the image looks nice, but because colab outputs it in its own font, it might looks bad after
          # being printed to the console.
          sliceOfImg = img[:maxHeight, x:x+dictionarySizes[key][0]]
          if sliceOfImg.shape[1] >= dictionarySizes[key][0]:
            try:
              mses.append([key, ssim(sliceOfImg, pieces[key])])
            except:
              pass
        # This checks to see if we have an error, which happens sometimes during testing
        if len(mses) != 0:
          # Gets the closest looking character
          argmin_mses = np.argmax(np.array(mses)[:,1])
          indexOfLowestMSE = mses[argmin_mses][0]
          # This is the only difference between this block and the else block, we add a space if the character is number 32, which is a space
          if indexOfLowestMSE == 32 and sendingToSnapchat:
            string += " "
          string += chr(indexOfLowestMSE)
        else:
          break
        x += dictionarySizes[indexOfLowestMSE][0]
      if x >= img.shape[1]:
        break
        
    # Add a new line and cut off the top line of the image
    img = img[maxHeight:, :]
    string += "\n"

### REFER TO OTHER BRANCH, THEY ARE ALMOST IDENTICAL
else:
  while True:
    print(round((1-(img.shape[0] / imgSize[0])) * 1000) * 0.1,"% Done")
    if img.shape[0] < maxHeight:
      break
    x = 0
    while True:
      mses = []
      for key in (dictionarySizes.keys()):
        sliceOfImg = img[:maxHeight, x:x+8]
        if sliceOfImg.shape[1] >= 8:
          try:
            mses.append([key, mse(sliceOfImg, pieces[key])])
          except:
            pass
      if len(mses) != 0:
        argmin_mses = np.argmin(np.array(mses)[:,1])
        indexOfLowestMSE = mses[argmin_mses][0]
        if indexOfLowestMSE == 32 and sendingToSnapchat:
          string += " "
        string += chr(indexOfLowestMSE)
        x += dictionarySizes[indexOfLowestMSE][0]
        if x >= img.shape[1]:
          break
      else:
        break
    img = img[maxHeight:, :]
    string += "\n"

# Prints string
print(string)